# ニュース記事分類

ライブドアニュースコーパス（ https://www.rondhuit.com/download.html )を分類します。同コーパスをダウンロードして解凍すると出てくるtextフォルダがこのノートと同じディレクトリにあると仮定します。

In [ ]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.decomposition import PCA
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import MeCab

mecab = MeCab.Tagger("-Ochasen -d /usr/lib/mecab/dic/mecab-ipadic-neologd/")

## データの読み込みと形態素解析

In [2]:
# コーパスのパス
path = "text/"

dirs = [i for i in os.listdir(path) if os.path.isdir(os.path.join(path, i))]
original = []
corpus = []
labels = []
label_to_class = {i:j for i,j in enumerate(dirs)}
for i, d in enumerate(tqdm(dirs)):
    p = os.path.join(path, d)
    files = [i for i in os.listdir(p) if i[-4:]==".txt" and i!="LICENSE.txt"]
    for f in files:
        with open(os.path.join(p, f), encoding="utf-8") as f:
            lines = f.readlines()[2:]
            lines = "".join(lines).replace("\n","")
            original.append(lines)
            node = mecab.parseToNode(lines)
            wakati = []
            while node:
                f = node.feature.split(",")[0]
                if f in ["名詞"]: # 今回は名詞のみ抽出した
                    wakati.append(node.surface)
                node = node.next
            wakati = " ".join(wakati)
            wakati
            corpus.append(wakati)
        labels.append(i)

100%|██████████| 9/9 [00:16<00:00,  1.82s/it]


## TF-IDFの計算

In [3]:
vectorizer = TfidfVectorizer(max_df=0.5, min_df=1, max_features=3000) # とりあえず3000次元とした
vector = vectorizer.fit_transform(corpus)
vector = vector.toarray()
features = vectorizer.get_feature_names()

## 次元削減

In [124]:
pca = PCA(n_components=100) # とりあえず100次元とした
red_vector = pca.fit_transform(vector)

(7367, 100)

## グリッドサーチ

In [139]:
parameters = {
    'C':[100, 200, 300, 400, 500, 600] #80
}
clf = GridSearchCV(svm.SVC(), parameters, n_jobs=-1)
clf.fit(red_vector, labels)
print(clf.best_params_)

pred = clf.predict(red_vector)
accuracy_score(labels, pred)

{'C': 500}


0.9421745622370028

## クロスバリデーション

In [140]:
scores = cross_val_score(clf, red_vector, labels, cv=5)
print("精度:", scores.mean(), "(+-", scores.std(), ")")

精度: 0.8809429403686003 (+- 0.038053964794232266 )


## 各カテゴリの代表文章抽出

In [4]:
labels_array = np.asarray(labels)
original = np.asarray(original)

for i, j in label_to_class.items():
    print("カテゴリ: ", j)
    idx = np.argmax(vector[labels_array==i].sum(axis=1))
    feature_idx = np.argsort(-vector[labels_array==i][idx])[:5]
    print("典型的な文章（特徴:", ",".join([features[i] for i in feature_idx]), ")")
    print(original[labels_array==i][idx][:100], "…")
    print("---------------")

カテゴリ:  dokujo-tsushin
典型的な文章（特徴: ガール,ネット,全員,友達,チェック )
森ガールよりもリア充？ 「雲ガール」の実態に迫る秋深き　隣は何を　する人ぞ　——。これまで「森ガール」にはじまり「山ガール」、「釣りガール」とさまざまな流行を生みだしてきた“○○ガールズ”ブーム。20 …
---------------
カテゴリ:  it-life-hack
典型的な文章（特徴: 機能,レビュー,リモコン,ホーム,設定 )
ようこそ、ケーブルのない世界へ！クリアな音質のBluetooth対応ヘッドセット【新スタイル活用術】「Bluetooth（ブルートゥース）」は、デジタル機器同士を接続するための無線規格だ。Blueto …
---------------
カテゴリ:  kaden-channel
典型的な文章（特徴: hd,制作,映像,レンジ,所有 )
【Special Report】ミドルレンジのビデオ制作者の有志が 集まって結成したHD Usersが選んだATEM 1M/E Switcher【ビデオSALON】“ミドルレンジ”といわれるビデオ制作 …
---------------
カテゴリ:  livedoor-homme
典型的な文章（特徴: スマホ,ストア,1月,mbps,for )
快適なスマホライフのための必須アプリ　昨年12月に発表されたユーキャン新語・流行語大賞でトップテンに選ばれ、日本人の5人に1人が持っているといわれる「スマホ」。MM総研の調査によると、2011年に国内 …
---------------
カテゴリ:  movie-enter
典型的な文章（特徴: 映画,公開,決定,貞子,アベンジャーズ )
【週末映画まとめ読み】注目するのは「貞子の野球カード」か「ボーイッシュ美少女の恋愛トーク」か　今週1週間に起こった出来事を振り返る「週末映画まとめ読み」。先週末の動員ランキングは『メン・イン・ブラック …
---------------
カテゴリ:  peachy
典型的な文章（特徴: 旅行,アート,アイテム,バッグ,便利 )
旅のおともはかわいくなくちゃ！(c) didi/amanaimages楽しい旅行に行くのに、“イマイチなモノ”と一緒じゃ気分が盛り上がらない！今回は、一緒に旅行に行きたい、かわいいトラベルグ